[![fraud-detection.gif](https://i.postimg.cc/zBtYZdf8/fraud-detection.gif)](https://postimg.cc/S2Mvcf8v)

<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 Welcome to this notebook. The goal of this notebook is to demonstrate how to handle credit card fraud datasets for ML fraud detection. Credit card fraud datasets have several specifications that make them unique and require specialized handling.
</div>

<div class="alert alert-block alert-danger" style="font-size:14px; font-family:verdana;">
     🚨 This notebook is written with separate code sections for ease of use. Each part of the code can be tested individually, facilitating experimentation and understanding.
</div>


## 1. Dataset Information

#### Context:
Credit card companies need to detect fraudulent transactions to prevent customers from being charged for unauthorized purchases.

#### Content:
- **Dataset**: Transactions made by European cardholders in September 2013.
- **Duration**: Two days, with 492 frauds out of 284,807 transactions.
- **Class Imbalance**: Fraudulent transactions (positive class) account for 0.172% of all transactions.
- **Features**:
  - Numerical input variables resulting from PCA transformation.
  - 'Time': Seconds elapsed between each transaction and the first transaction.
  - 'Amount': Transaction amount, suitable for cost-sensitive learning.

- **Target**:
  - 'Class': Response variable, 1 for fraud, 0 otherwise.
  
#### Source:
- The dataset has been collected and analyzed by Worldline and the Machine Learning Group (MLG) of Université Libre de Bruxelles (ULB) as part of a research collaboration on big data mining and fraud detection.
- More details on the current and past projects related to fraud detection are available on the [MLG website](http://mlg.ulb.ac.be) and [ResearchGate](https://www.researchgate.net/project/Fraud-detection-5).

#### Recommendations:
- Due to class imbalance, accuracy should be measured using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful.
  




## 2. Exploratory Data Analysis

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read the CSV file 'creditcard.csv' into a Pandas DataFrame named df
df = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')


In [ ]:
# Display the first few rows of the DataFrame df
df.head()

In [ ]:
# Display a concise summary of the DataFrame df, including column names, non-null counts...
df.info()


In [ ]:
# Print the shape of the dataset (number of rows and columns)
print('Shape Of The Dataset', df.shape)

# Print the unique class categories in the 'Class' column
print('Class Categories', df['Class'].unique())

# Print the number of records with the class value 0 in the 'Class' column
print('Number Of Records With The Class Value 0: ', (df.Class == 0).sum())

# Print the number of records with the class value 1 in the 'Class' column
print('Number Of Records With The Class Value 1: ', (df.Class == 1).sum())


In [ ]:
# Create a count plot to visualize the distribution of classes in the 'Class' column of the DataFrame df
sns.countplot(x='Class', data=df)


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    ⚠️ Credit card fraud datasets, including this one, are typically highly imbalanced because occurrences of fraud are rare compared to normal transactions. In the next sections, we will explore effective strategies for handling this imbalance.
</div>

## 3. Features Selection

In [ ]:
# Calculate the correlation between the 'Class' column and the first 30 columns 
x = df.corr()['Class'][:30]
x

In [ ]:
# Calculate the correlation coefficients between the 'Class' column and the first 30 columns 
x = df.corr()['Class'][:30]

# Create a bar plot to visualize the correlation of features with the target variable 'Class'
x.plot.bar(figsize=(16, 9), title="Correlation Of Features With Target Variable", grid=True)

<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 Some features exhibit a negligible correlation with the target variable and will be removed in the subsequent sections. First, we'll examine the intercorrelation among variables.
</div>

In [ ]:
# Create a figure with a specific size for the heatmap
plt.figure(figsize=(16, 9))

# Create a heatmap to visualize the correlation matrix of the DataFrame df
sns.heatmap(df.corr())

<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 The only intercorrelated variable among others is the transaction Amount. However, this variable shows no correlation with the target variable, so it will also be removed.
</div>

In [ ]:
# Calculate the correlation coefficients between 'Class' and all columns
y = df.corr()['Class']

# Create a copy of the DataFrame df
df2 = df.copy()

# Iterate through columns and drop those with absolute correlation less than 0.13
for i in df.columns:
    if abs(y[i]) < 0.13:
        df2.drop(columns=[i], inplace=True)


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌Here, we filter our dataset to keep only features with a correlation above 0.13.
</div>

In [ ]:
# Display the first few rows of the DataFrame df2
df2.head()

In [ ]:
# Create a figure with a specific size for the heatmap
plt.figure(figsize=(16, 9))

# Create a heatmap to visualize the correlation matrix of the DataFrame df2
sns.heatmap(df2.corr(), annot=True)

In [ ]:
# Calculate the correlation coefficients between the 'Class' column 
x = df2.corr()['Class'][:9]

# Create a bar plot to visualize the top correlated features with the target variable 'Class'
x.plot.bar(figsize=(16, 9), title="Top Correlated Features With The Target Variable", grid=True)


## 4. Handling Data Imbalance



- This dataset consists of:
  - Number of records with the class value 0: 284,315
  - Number of records with the class value 1: 492

Using this dataset as it is would be a fatal mistake due to its severe class imbalance. Here's why:

- **Using the data as it is:**
  - The overwhelming majority of records belong to the non-fraudulent class (class 0), making up over 99% of the dataset.
  - Models trained on imbalanced data may prioritize accuracy on the majority class while neglecting the minority class (fraudulent transactions). This can result in poor performance in detecting fraud.

- **Why oversampling is a fatal mistake:**
  - Oversampling techniques like SMOTE (Synthetic Minority Over-sampling Technique) artificially inflate the minority class by generating synthetic examples. However, this can lead to overfitting and the introduction of noise, especially in cases where the minority class is already sparsely represented.

- **Why downsampling is the best option:**
  - Downsampling involves randomly reducing the number of samples in the majority class to balance it with the minority class. This approach helps mitigate the biases towards the majority class while maintaining the integrity of the dataset.
  - By reducing the number of majority class samples to match the minority class, downsampling encourages the model to learn from both classes equally, improving its ability to accurately detect fraudulent transactions.




1. **Using Imbalanced Data (No Sampling) Example:**
   - Dataset:
     - Class 0 (non-fraudulent transactions): 284,315 records
     - Class 1 (fraudulent transactions): 492 records
   - Example:
     - Accuracy on test set: 99.8%
     - Confusion Matrix:
       ```
                 Predicted Non-Fraudulent    Predicted Fraudulent
       Actual Non-Fraudulent      71,078               200
       Actual Fraudulent              50                40
       ```
     - Issue: High accuracy is misleading; the model fails to detect most fraudulent transactions (low recall for class 1).

2. **Oversampling (SMOTE) Example:**
   - Dataset:
     - Original Class 0: 284,315 records
     - Class 1: 492 records
     - After SMOTE (oversampling Class 1 to match Class 0):
       - Class 0: 284,315 records
       - Class 1: 284,315 records (synthetic)
   - Example:
     - Model Performance:
       - Accuracy: 98.5%
       - Confusion Matrix:
         ```
                   Predicted Non-Fraudulent    Predicted Fraudulent
         Actual Non-Fraudulent      70,800                  478
         Actual Fraudulent              10                   80
         ```
     - Issue: High accuracy but high false positives due to synthetic examples, leading to overfitting and reduced precision for fraud detection.

<div class="alert alert-block alert-danger" style="font-size:14px; font-family:verdana;">
     ⚠️ Please note that we are discussing the training data, not the test data. The test data should represent "reality." If the data the model will encounter in real life is imbalanced, then the model should be tested on imbalanced data. However, as explained earlier, to avoid bias in learning, the model should be trained on balanced data.

</div>


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 Having recognized why downsampling is the optimal technique for this dataset, let's proceed with downsampling our dataset.
    We will downsample the entire dataset just to study outliers. Later, we will perform a proper split of the training and test data. The training data will be downsampled/balanced, while the test data will reflect the original data distribution to evaluate the model in a real-world scenario
</div>

In [ ]:

from imblearn.under_sampling import RandomUnderSampler

# Separate features (X) and target (y)
X = df2.drop('Class', axis=1)
y = df2['Class']

# Initialize RandomUnderSampler
rus = RandomUnderSampler(random_state=42)

# Fit and apply the resampler to the data
X_resampled, y_resampled = rus.fit_resample(X, y)

# Convert the resampled data back to a DataFrame
downsampled_df = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=['Class'])], axis=1)


downsampled_df.head()

In [ ]:
# Display the shape of the downsampled DataFrame downsampled_df
downsampled_df.shape

## 5. Outliers?

In [ ]:
# Create a count plot to visualize the distribution of classes in the 'Class' column of the downsampled DataFrame downsampled_df
sns.countplot(x='Class', data=downsampled_df)


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 Now that our dataset is balanced, we can move on to the next section: handling outliers. How should we approach outliers? Should we simply delete them? Let's explore.
</div>

In [ ]:
# Plotting using seaborn scatterplot
sns.scatterplot(x='V11', y='V17', hue='Class', data=df2)


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 Outliers are data points that significantly deviate from the average of a variable. In other words, they do not conform to the typical grouping observed in a specific cluster. Did you notice something interesting in this plot? The blue dots represent normal transactions, tightly clustered with very few outliers. In contrast, the orange dots represent fraudulent transactions, which do not form a distinct cluster, making outlier detection challenging. Is this pattern consistent across all variables or just in this example? Let's investigate further.
</div>

In [ ]:
import warnings

# To ignore all warnings
warnings.filterwarnings('ignore')
# Pair Plot of all variables
sns.pairplot(downsampled_df, hue='Class')

<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 As you may have observed in this pairplot, normal transactions form a distinct cluster across all variables, whereas fraudulent transactions do not exhibit a specific cluster. This makes outlier detection extremely challenging. Attempting to manage outliers individually for each variable could result in transforming or deleting over 70% of the fraudulent transactions.
</div>

<div class="alert alert-block alert-danger" style="font-size:14px; font-family:verdana;">
     ⚠️ Fraudulent transactions do not exhibit clustering behavior and do not conform to a normal distribution, making outlier identification challenging.
    

<br>- This phenomenon arises because fraud does not adhere to a typical distribution pattern; in other words, fraudulent activities vary widely and do not consistently follow specific patterns.

<br>- Fraudsters employ diverse methods that evolve over time. It's important to note that simply obtaining credit card details and full names is insufficient for completing transactions. Fraudsters often employ additional techniques such as SIM swapping to bypass payment verification processes.

<br>- Due to these factors, datasets containing fraudulent transactions lack distinct clustering and do not adhere to a normal distribution.

<br>For these reasons, no records will be deleted from fraudulent transactions. Additionally, it's important to consider that fraudulent transactions are rare, making each record valuable. 

<br>Note: Outliers can still be removed from normal transactions (non-fraudulent transactions).
</div>


## 6. Proper Data Splitting

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix

# Original imbalanced data (df2)
X = df2.drop(columns='Class')
y = df2['Class']

# Step 1: Split the original data into training and testing sets with stratification
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 2: Downsample only the training data to ensure no data leakage
train_data = X_train_orig.copy()
train_data['Class'] = y_train_orig

# Separate the majority and minority classes in the training data
majority_class = train_data[train_data['Class'] == 0]  # Assuming 0 is the majority class
minority_class = train_data[train_data['Class'] == 1]  # Assuming 1 is the minority class

# Downsample the majority class
majority_downsampled = resample(majority_class, 
                                replace=False,    # sample without replacement
                                n_samples=len(minority_class),  # match minority class size
                                random_state=42)  # reproducible results

# Combine the downsampled majority class with the minority class
downsampled_train_data = pd.concat([majority_downsampled, minority_class])

# Step 3: Prepare the downsampled training set and the original test set
X_train_downsampled = downsampled_train_data.drop(columns='Class')
y_train_downsampled = downsampled_train_data['Class']

<div class="alert alert-block alert-danger" style="font-size:14px; font-family:verdana;">
     ⚠️ To properly split the data, we need to follow these rules:

- **Use a stratified split:** Since the data is highly imbalanced, a random split might result in a training or test set with no fraud transactions. Fraud transactions represent only a fraction of the dataset, so the risk of ending up with a dataset without fraud transactions is high. A stratified split ensures that this imbalance is maintained in both the training and test sets, preventing this issue.

- **Downsample the training data:** As previously explained, this is important to avoid bias and ensure that the model learns effectively.

- **Ensure the distribution of the test data matches the distribution in the original data:** This is crucial for evaluating the model's performance in real-world scenarios.

- **Avoid data leakage:** Ensure that there is no overlap between the final training and test datasets to prevent information from the test set influencing the training process.

</div>

## 7. Quick Test Using LazyPredict

In [ ]:
!pip install lazypredict


In [ ]:
from lazypredict.Supervised import LazyClassifier

# Step 4: Initialize LazyClassifier and fit the model
clf = LazyClassifier(random_state=42)

# Fit the models using the downsampled training data and test on the original test set
models, predictions = clf.fit(X_train_downsampled, X_test_orig, y_train_downsampled, y_test_orig)

# Display the performance of the models
models

<div class="alert alert-block alert-danger" style="font-size:14px; font-family:verdana;">
     ⚠️ **Avoiding the Big Trap:** Let's remember that our test data is highly imbalanced. Relying solely on accuracy as the main metric for choosing our model is a significant mistake. The proper approach is to find a model that balances accuracy, precision, recall, and F1 score. However, since detecting normal transactions is not challenging for machine learning models, we will focus on the F1 score for fraudulent transactions (Class == 1).

</div>

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import NuSVC
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import f1_score

# Initialize the models
models = {
    'BernoulliNB': BernoulliNB(),
    'NuSVC': NuSVC(probability=True),  # NuSVC requires probability=True for probability estimates
    'NearestCentroid': NearestCentroid()
}

# Fit models and calculate F1 scores
f1_scores = {}

for name, model in models.items():
    # Fit the model
    model.fit(X_train_downsampled, y_train_downsampled)
    
    # Make predictions
    y_pred = model.predict(X_test_orig)
    
    # Calculate F1 score for class == 1
    f1_scores[name] = f1_score(y_test_orig, y_pred, pos_label=1)

# Display the F1 scores for each model
print("F1 Scores for class == 1:")
for name, score in f1_scores.items():
    print(f"{name}: {score:.4f}")


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 Here, we chose the three models with the highest accuracy. However, we compared their F1 scores for Class == 1 and obtained the following results:

**F1 Scores for Class == 1:**
- BernoulliNB: 0.0994
- NuSVC: 0.5950
- NearestCentroid: 0.8144

Our winner is NearestCentroid, so let's proceed to the next section.
</div>

## 8. Final Model

In [ ]:
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


# Initialize the NearestCentroid model
FRAUDFIGHTER = NearestCentroid()

# Fit the model on the downsampled training data
FRAUDFIGHTER.fit(X_train_downsampled, y_train_downsampled)

# Predict on the original test data
y_pred = FRAUDFIGHTER.predict(X_test_orig)

# Generate classification report
report = classification_report(y_test_orig, y_pred)
print("Classification Report:\n", report)

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test_orig, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-Fraud', 'Fraud'], 
            yticklabels=['Non-Fraud', 'Fraud'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 FRAUDFIGHTER demonstrates excellent performance in identifying normal transactions and shows strong, though slightly less perfect, performance in detecting fraudulent transactions. The high accuracy is somewhat skewed by the imbalanced dataset, but the F1 score for fraudulent transactions indicates that the model is effective at detecting fraud while maintaining a good balance between precision and recall.
</div>

## 9. Minimizing False Negatives: A Crucial Step

<div class="alert alert-block alert-danger" style="font-size:14px; font-family:verdana;">
     ⚠️ In credit card fraud detection, minimizing false negatives (fraudulent transactions classified as non-fraudulent) is crucial and often as important as accuracy, if not more so. Classifying a fraudulent transaction as non-fraudulent can pose significant risks, potentially leading to financial losses and undermining trust in the detection system. The consequences of missing fraudulent transactions can be severe, as they may go undetected and lead to further fraudulent activities. Therefore, achieving a balance between high accuracy and low false negatives is essential for robust fraud detection systems.

</div>


<div class="alert alert-block alert-danger" style="font-size:14px; font-family:verdana;">
     ⚠️ Example: 
    


- Suppose a bank earns $1 profit per transaction, whether normal or fraudulent.
- There are 100 transactions: 99 normal and 1 fraudulent.
- Profit from 99 normal transactions: ( 99 x \$1 = \$99 )

Now, consider the fraudulent transaction:
- Fraudulent transaction amount: \$100.
- If misclassified as normal, bank earns: \$1.
- Actual loss to the bank: \$100.

In this case:
- Bank's profit from normal transactions: \$99.
- Loss from misclassified fraudulent transaction: \$100.

Misclassifying the fraudulent transaction as normal means:
- Bank's total profit calculation: \( \$99 + \$1 = \$100 \).
- Actual loss due to fraud: \$100.

Therefore, misclassifying one fraudulent transaction as normal would result in the bank losing all the profits earned from normal transactions, resulting in a net profit of \$0.

</div>

<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 For this dataset, we didn’t find a way to reduce false negatives. However, we included the above disclaimers to make this notebook useful for all kinds of datasets. We also explained why false negatives are a significant issue for fraud detection datasets!
</div>

## 10. Features Importance

In [ ]:
from sklearn.inspection import permutation_importance


# Compute permutation importance
result = permutation_importance(model, X_test_orig, y_test_orig, n_repeats=10, random_state=42, scoring='f1')

# Get mean and standard deviation of importance values
importance = result.importances_mean
std = result.importances_std

# Create a DataFrame for easy plotting
importance_df = pd.DataFrame({
    'Feature': X_test_orig.columns,
    'Importance': importance,
    'Std Dev': std
}).sort_values(by='Importance', ascending=False)

# Print the DataFrame
print("Feature Importances:\n", importance_df)

# Plot feature importances
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=importance_df, xerr=importance_df['Std Dev'], palette='viridis')
plt.xlabel('Mean Importance')
plt.ylabel('Feature')
plt.title('Permutation Importance of Features')
plt.show()


<div class="alert alert-block alert-secondary" style="font-size:14px; font-family:verdana; background-color:#d3d3d3; color:#555555;">
    📌 We assessed feature importance and found that `V3`, `V14`, and `V17` are the top contributors to our model’s performance. 
</div>

## 11. Conclusions 


1. **Effective Model Development:** Through meticulous data preprocessing, feature selection, and model optimization steps, we've developed a robust fraud detection model.
   
2. **Importance of Imbalance Handling:** Addressing the imbalance in the dataset was critical. Downsampling the majority class improved model performance by ensuring balanced representation of fraudulent and non-fraudulent transactions.

3. **Model Performance:** Our final model, FRAUDFIGHTER, achieved an impressive accuracy of 99% with minimal false negatives and false positives. This underscores its capability to accurately detect fraudulent transactions.

6. **Real-world Application:** The methodologies and techniques applied here are crucial for real-world applications, where the cost of misclassifying fraudulent transactions can be substantial both financially and in terms of trust and customer satisfaction.

7. **Future Directions:** Further enhancements could involve exploring advanced feature engineering techniques, integrating additional data sources, or deploying the model in a real-time environment to continuously improve fraud detection capabilities.

<div class="alert alert-block alert-info" style="font-size:14px; font-family:verdana;">
    📌 👋 Thank you for reading this notebook! If you found this content useful, please consider giving it an upvote. Your support is greatly appreciated! 🌟.
</div>